In [ ]:
from datetime import datetime

import xarray as xr
import matplotlib.pyplot as plt

from gfatpy.lidar.preprocessing import preprocess

from gfatpy.atmo.ecmwf import get_ecmwf_temperature_pressure
from gfatpy.atmo.freudenthaler_molecular_properties import molecular_properties

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
nc = preprocess("mhc_1a_Prs_rs_xf_20221027.nc")

In [ ]:
(nc.signal_532xpa[-5] * nc.range ** 2).plot()

In [ ]:
signal_mean = nc.signal_1064xta[-30:].mean("time")
rcs_mean = signal_mean * nc.range ** 2

In [ ]:
rcs_mean

In [ ]:
atmo = get_ecmwf_temperature_pressure("2022-10-26", hour=14, heights=rcs_mean.range)
mol_props = molecular_properties(
    1064, atmo["pressure"].values, atmo["temperature"].values, heights=rcs_mean.range
)

In [ ]:
mol_props.attenuated_molecular_beta

In [ ]:
from scipy.signal import savgol_filter

fig, ax = plt.subplots()

rcs_mean_n = rcs_mean / rcs_mean.sel(range=7000, method="nearest")
att_beta_n = mol_props.attenuated_molecular_beta / mol_props.attenuated_molecular_beta.sel(range=7000, method="nearest")

ax.plot(rcs_mean_n.range, rcs_mean_n.values, c = "lightcoral", alpha = 0.6, label = "Raw RCS")
ax.plot(rcs_mean_n.range, savgol_filter(rcs_mean_n.values, 11, 3), c = "red", label = "Smooth RCS")
ax.plot(att_beta_n.range, att_beta_n.values, c = "k", label = "Attenuated backscatter")

ax.set(xlabel="Height a.g.l [m]", ylabel="RCS normalized")
ax.legend()

ax.set_yscale("log")
fig.savefig("1064-fixed-rayleigh.png", dpi = 600)

In [ ]:
raw_nc = xr.load_dataset("../tests/datos/MULHACEN/1a/2022/08/08/mhc_1a_Prs_rs_xf_20220808_1131.nc")
raw_rcs = raw_nc.signal_355xta[0] * raw_nc.range ** 2

In [ ]:
raw_rcs.time

In [ ]:
nc = preprocess("mhc_1a_Prs_rs_xf_20210725.nc", apply_dc=True)
nc_nodc = preprocess("mhc_1a_Prs_rs_xf_20210725.nc", apply_dc=False)


rcs = nc.signal_355xta[130] * nc.range ** 2
rcs_nodc = nc_nodc.signal_355xta[130] * nc.range ** 2

In [ ]:
nc.time[130]

In [ ]:
atmo = get_ecmwf_temperature_pressure("2021-07-25", hour=2, heights=rcs.range)
mol_props = molecular_properties(
    355, atmo["pressure"].values, atmo["temperature"].values, heights=rcs.range
)

In [ ]:
rcs.plot()

In [ ]:
from scipy.signal import savgol_filter

fig, ax = plt.subplots(figsize=(15, 7))

rcs_mean_n = rcs / rcs.sel(range=slice(6900, 7000)).mean("range")
rcs_mean_n_nodc = rcs_nodc / rcs.sel(range=slice(6900, 7000)).mean("range")
att_beta_n = mol_props.attenuated_molecular_beta / mol_props.attenuated_molecular_beta.sel(range=slice(6900, 7000)).mean("range")

# ax.plot(rcs.range, rcs_mean_n.values, c = "skyblue", alpha = 0.6, label = "Raw RCS")
ax.plot(rcs_mean_n.range, savgol_filter(rcs_mean_n.values, 11, 3), c = "blue", label = "DC-corrected smoothed RCS", lw = 0.6)
ax.plot(rcs_mean_n.range, savgol_filter(rcs_mean_n_nodc.values, 11, 3), c = "red", label = "DC-uncorrected smoothed RCS", lw = 0.6)
ax.plot(att_beta_n.range, att_beta_n.values, c = "k", label = "Attenuated backscatter")

ax.set(xlabel="Height a.g.l [m]", ylabel="RCS normalized")
ax.set_yscale("log")
ax.legend()

fig.savefig("355-dc-comparison.png", dpi = 600)

In [ ]:
from scipy.signal import savgol_filter

fig, ax = plt.subplots(figsize=(15, 7))

rcs_mean_n = rcs_nodc / rcs.sel(range=slice(6500, 7000)).mean("range")
att_beta_n = mol_props.attenuated_molecular_beta / mol_props.attenuated_molecular_beta.sel(range=slice(6900, 7000)).mean("range")

ax.plot(rcs.range, rcs_mean_n.values, c = "skyblue", alpha = 0.6, label = "Raw RCS")
ax.plot(rcs_mean_n.range, savgol_filter(rcs_mean_n.values, 11, 3), c = "blue", label = "Smooth RCS")
ax.plot(att_beta_n.range, att_beta_n.values, c = "k", label = "Attenuated backscatter")

ax.set(xlabel="Height a.g.l [m]", ylabel="RCS normalized")
ax.set_yscale("log")
ax.legend()